# NYS REGENTS Highschool State Examination Scores
## An analysis and regression of scores from 2014-2019
### Author: Jack Robbins

In [119]:
# Important imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
from sklearn import preprocessing
from matplotlib.gridspec import GridSpec

In [120]:
# Read in our dataframe
scores = pd.read_csv("data/2014-15_to_2017-19_NYC_Regents_Exam_Results_-_Public_20241026.csv")

In [121]:
scores

,Unnamed Column,School DBN,School Name,School Type,School Level,Regents Exam,Year,Category,Total Tested,Mean Score,Number Scoring Below 65,Percent Scoring Below 65,Number Scoring 65 or Above,Percent Scoring 65 or Above,Number Scoring 80 or Above,Percent Scoring 80 or Above,Number Scoring CR,Percent Scoring CR
0,08X5072015All StudentsGeometry,08X507,Stevenson Yabc,YABC,High school,Geometry,2015,All Students,1,s,s,s,s,s,s,s,s,s
1,01M1842018All StudentsLiving Environment,01M184,P.S. 184m Shuang Wen,General Academic,K-8,Living Environment,2018,All Students,32,90.2,0,0.0,32,100.0,31,96.9,na,na
2,01M4582019All StudentsSpanish,01M458,Forsyth Satellite Academy,Transfer School,High school,Spanish,2019,All Students,1,s,s,s,s,s,s,s,s,s
3,01M5092017All StudentsSpanish,01M509,Marta Valle High School,General Academic,High school,Spanish,2017,All Students,16,82.7,0,0.0,16,100.0,10,62.5,na,na
4,02M2982015All StudentsGeometry,02M298,Pace High School,General Academic,High school,Geometry,2015,All Students,163,62.9,81,49.7,82,50.3,13,8.0,13,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33026,84X7172017All StudentsLiving Environment,84X717,Icahn Charter School,General Academic,K-8,Living Environment,2017,All Students,10,79.7,0,0.0,10,100.0,5,50.0,na,na
33027,84X7172018All StudentsCommon Core Algebra,84X717,Icahn Charter School,General Academic,K-8,Common Core Algebra,2018,All Students,15,86.9,0,0.0,15,100.0,15,100.0,15,100.0
33028,84X7172018All StudentsLiving Environment,84X717,Icahn Charter School,General Academic,K-8,Living Environment,2018,All Students,9,85.3,0,0.0,9,100.0,8,88.9,na,na
33029,84X7172019All StudentsCommon Core Algebra,84X717,Icahn Charter School,General Academic,K-8,Common Core Algebra,2019,All Students,15,93.2,0,0.0,15,100.0,15,100.0,15,100.0


## Data Preprocessing
As we can see from above, this data is quite a mess with unnamed columns, weird concatenations and other issues. We'll have to clean it up before using it

### Dropping Columns
The following columns are either meaningless or it is not documented as to what they even are, they are as follows:
- Unnamed Column: appears to be a concatenation of "category", "school dbn" and the REGENTS exam type
- School DBN/Name: these are both unique identifiers, and really won't help us much
- Number & percent scoring CR: it is completely undocumented as to what these columns even mean. As such they won't be a help to us

These columns will all be dropped just to clear up the noise in this dataset

In [122]:
# Dropping the mentioned columns
scores.drop(['Unnamed Column', 'School DBN', 'School Name', 'Number Scoring CR', 'Percent Scoring CR'], axis=1, inplace=True)
scores

,School Type,School Level,Regents Exam,Year,Category,Total Tested,Mean Score,Number Scoring Below 65,Percent Scoring Below 65,Number Scoring 65 or Above,Percent Scoring 65 or Above,Number Scoring 80 or Above,Percent Scoring 80 or Above
0,YABC,High school,Geometry,2015,All Students,1,s,s,s,s,s,s,s
1,General Academic,K-8,Living Environment,2018,All Students,32,90.2,0,0.0,32,100.0,31,96.9
2,Transfer School,High school,Spanish,2019,All Students,1,s,s,s,s,s,s,s
3,General Academic,High school,Spanish,2017,All Students,16,82.7,0,0.0,16,100.0,10,62.5
4,General Academic,High school,Geometry,2015,All Students,163,62.9,81,49.7,82,50.3,13,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33026,General Academic,K-8,Living Environment,2017,All Students,10,79.7,0,0.0,10,100.0,5,50.0
33027,General Academic,K-8,Common Core Algebra,2018,All Students,15,86.9,0,0.0,15,100.0,15,100.0
33028,General Academic,K-8,Living Environment,2018,All Students,9,85.3,0,0.0,9,100.0,8,88.9
33029,General Academic,K-8,Common Core Algebra,2019,All Students,15,93.2,0,0.0,15,100.0,15,100.0


### Rows with few students who took the exam
Looking at the data above reveals that there are some rows that have schools/tests where 1 student took the exam. These are not of much interest to us, so we'll remove any row where less than 10 students took the exam.

In [123]:
indices_to_drop = scores[scores["Total Tested"] < 10].index
scores.drop(indices_to_drop, axis=0, inplace=True)

In [124]:
scores

,School Type,School Level,Regents Exam,Year,Category,Total Tested,Mean Score,Number Scoring Below 65,Percent Scoring Below 65,Number Scoring 65 or Above,Percent Scoring 65 or Above,Number Scoring 80 or Above,Percent Scoring 80 or Above
1,General Academic,K-8,Living Environment,2018,All Students,32,90.2,0,0.0,32,100.0,31,96.9
3,General Academic,High school,Spanish,2017,All Students,16,82.7,0,0.0,16,100.0,10,62.5
4,General Academic,High school,Geometry,2015,All Students,163,62.9,81,49.7,82,50.3,13,8.0
7,General Academic,High school,French,2019,All Students,21,81.5,0,0.0,21,100.0,11,52.4
8,General Academic,High school,English,2015,All Students,128,87.4,2,1.6,126,98.4,112,87.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33024,General Academic,K-8,Common Core Algebra,2017,All Students,12,87.7,0,0.0,12,100.0,11,91.7
33026,General Academic,K-8,Living Environment,2017,All Students,10,79.7,0,0.0,10,100.0,5,50.0
33027,General Academic,K-8,Common Core Algebra,2018,All Students,15,86.9,0,0.0,15,100.0,15,100.0
33029,General Academic,K-8,Common Core Algebra,2019,All Students,15,93.2,0,0.0,15,100.0,15,100.0


So we've now got data where 10 or more students took the exam

### Looking for bad values using unique()
We can use the unique() function to find any bad data in our categorical columns pretty easily. If we find it, we can action upon it

In [125]:
# Let's test the year. It looks like we're all clear for this here too.
scores["Year"].unique()

array([2018, 2017, 2015, 2019, 2016])

In [126]:
# Let's test the School Type. Looks all good to me
scores["School Type"].unique()

array(['General Academic', 'Transfer School', 'Special Education',
       'Career Technical', 'YABC'], dtype=object)

In [127]:
# Let's test the school level
scores["School Level"].unique()

array(['K-8', 'High school', 'Junior High-Intermediate-Middle',
       'Secondary School', 'K-12 all grades', 'Elementary', 'Ungraded'],
      dtype=object)

### Dealing with weird values in School Level
As we can see we do have some weird values in here. We will eliminate the Secondary School option as this is just highschool, and we will also remove the ungraded school as well as this won't help us in a later analysis.

In [128]:
indices_to_drop = scores[~(scores["School Level"].isin(['K-8', 'High school', 'Junior High-Intermediate-Middle', 'K-12 all grades', 'Elementary']))].index
scores.drop(indices_to_drop, axis=0, inplace=True)
scores

,School Type,School Level,Regents Exam,Year,Category,Total Tested,Mean Score,Number Scoring Below 65,Percent Scoring Below 65,Number Scoring 65 or Above,Percent Scoring 65 or Above,Number Scoring 80 or Above,Percent Scoring 80 or Above
1,General Academic,K-8,Living Environment,2018,All Students,32,90.2,0,0.0,32,100.0,31,96.9
3,General Academic,High school,Spanish,2017,All Students,16,82.7,0,0.0,16,100.0,10,62.5
4,General Academic,High school,Geometry,2015,All Students,163,62.9,81,49.7,82,50.3,13,8.0
7,General Academic,High school,French,2019,All Students,21,81.5,0,0.0,21,100.0,11,52.4
8,General Academic,High school,English,2015,All Students,128,87.4,2,1.6,126,98.4,112,87.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33024,General Academic,K-8,Common Core Algebra,2017,All Students,12,87.7,0,0.0,12,100.0,11,91.7
33026,General Academic,K-8,Living Environment,2017,All Students,10,79.7,0,0.0,10,100.0,5,50.0
33027,General Academic,K-8,Common Core Algebra,2018,All Students,15,86.9,0,0.0,15,100.0,15,100.0
33029,General Academic,K-8,Common Core Algebra,2019,All Students,15,93.2,0,0.0,15,100.0,15,100.0


In [129]:
# Let's test the regents exam as well. This looks to be in the clear to me
scores["Regents Exam"].unique()

array(['Living Environment', 'Spanish', 'Geometry', 'French', 'English',
       'Common Core Algebra', 'Common Core English',
       'Common Core Geometry', 'Global History and Geography',
       'Integrated Algebra', 'Physical Settings/Earth Science',
       'U.S. History and Government', 'Common Core Algebra2',
       'Physical Settings/Chemistry', 'Algebra2/Trigonometry', 'Chinese',
       'Physical Settings/Physics', 'Italian'], dtype=object)

In [130]:
# Let's finally test the category
scores["Category"].unique()

array(['All Students'], dtype=object)

### A surprise find
I'm glad that I did this. As we can see, this column is completely useless because every single value is 'All Students'. We can drop it as a result.

In [131]:
scores.drop('Category', axis=1, inplace=True)
scores

,School Type,School Level,Regents Exam,Year,Total Tested,Mean Score,Number Scoring Below 65,Percent Scoring Below 65,Number Scoring 65 or Above,Percent Scoring 65 or Above,Number Scoring 80 or Above,Percent Scoring 80 or Above
1,General Academic,K-8,Living Environment,2018,32,90.2,0,0.0,32,100.0,31,96.9
3,General Academic,High school,Spanish,2017,16,82.7,0,0.0,16,100.0,10,62.5
4,General Academic,High school,Geometry,2015,163,62.9,81,49.7,82,50.3,13,8.0
7,General Academic,High school,French,2019,21,81.5,0,0.0,21,100.0,11,52.4
8,General Academic,High school,English,2015,128,87.4,2,1.6,126,98.4,112,87.5
...,...,...,...,...,...,...,...,...,...,...,...,...
33024,General Academic,K-8,Common Core Algebra,2017,12,87.7,0,0.0,12,100.0,11,91.7
33026,General Academic,K-8,Living Environment,2017,10,79.7,0,0.0,10,100.0,5,50.0
33027,General Academic,K-8,Common Core Algebra,2018,15,86.9,0,0.0,15,100.0,15,100.0
33029,General Academic,K-8,Common Core Algebra,2019,15,93.2,0,0.0,15,100.0,15,100.0


In [132]:
# Do we have any Null values?
null_values=scores.isnull().sum()
print("Detecting missing values:\n", null_values)

Detecting missing values:
 School Type                    0
School Level                   0
Regents Exam                   0
Year                           0
Total Tested                   0
Mean Score                     0
Number Scoring Below 65        0
Percent Scoring Below 65       0
Number Scoring 65 or Above     0
Percent Scoring 65 or Above    0
Number Scoring 80 or Above     0
Percent Scoring 80 or Above    0
dtype: int64


So there are no "null" values here, but that is deceptive. It seems that whomever entered these data actually typed in "na" wherever they meant to write null. As such, we'll have to clean these out in the data.

In [133]:
scores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24147 entries, 1 to 33030
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   School Type                  24147 non-null  object
 1   School Level                 24147 non-null  object
 2   Regents Exam                 24147 non-null  object
 3   Year                         24147 non-null  int64 
 4   Total Tested                 24147 non-null  int64 
 5   Mean Score                   24147 non-null  object
 6   Number Scoring Below 65      24147 non-null  object
 7   Percent Scoring Below 65     24147 non-null  object
 8   Number Scoring 65 or Above   24147 non-null  object
 9   Percent Scoring 65 or Above  24147 non-null  object
 10  Number Scoring 80 or Above   24147 non-null  object
 11  Percent Scoring 80 or Above  24147 non-null  object
dtypes: int64(2), object(10)
memory usage: 2.4+ MB


### Converting numerical columns into numbers
As we can see above, columns that should be numeric are actually registered as objects. This won't help us in our regression, so we'll need to fix this.

In [134]:
scores["Mean Score"] = scores["Mean Score"].apply(lambda x: float(x))
scores["Number Scoring Below 65"] = scores["Number Scoring Below 65"].apply(lambda x: int(x))
scores["Percent Scoring Below 65"] = scores["Percent Scoring Below 65"].apply(lambda x: float(x))
scores["Number Scoring 65 or Above"] = scores["Number Scoring 65 or Above"].apply(lambda x: int(x))
scores["Percent Scoring 65 or Above"] = scores["Percent Scoring 65 or Above"].apply(lambda x: float(x))
scores["Number Scoring 80 or Above"] = scores["Number Scoring 80 or Above"].apply(lambda x: int(x))
scores["Percent Scoring 80 or Above"] = scores["Percent Scoring 80 or Above"].apply(lambda x: float(x))

scores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24147 entries, 1 to 33030
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   School Type                  24147 non-null  object 
 1   School Level                 24147 non-null  object 
 2   Regents Exam                 24147 non-null  object 
 3   Year                         24147 non-null  int64  
 4   Total Tested                 24147 non-null  int64  
 5   Mean Score                   24147 non-null  float64
 6   Number Scoring Below 65      24147 non-null  int64  
 7   Percent Scoring Below 65     24147 non-null  float64
 8   Number Scoring 65 or Above   24147 non-null  int64  
 9   Percent Scoring 65 or Above  24147 non-null  float64
 10  Number Scoring 80 or Above   24147 non-null  int64  
 11  Percent Scoring 80 or Above  24147 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 2.4+ MB


### Encoding Categorical Data
Now that the data has been cleaned, we'll need to encode our 3 categorical columns for use in our regression

In [135]:
school_type_le = preprocessing.LabelEncoder()
school_level_le = preprocessing.LabelEncoder()
regents_exam_le = preprocessing.LabelEncoder()

scores['School Type'] = school_type_le.fit_transform(scores['School Type'])
scores['School Level'] = school_level_le.fit_transform(scores['School Level'])
scores['Regents Exam'] = regents_exam_le.fit_transform(scores['Regents Exam'])

scores

,School Type,School Level,Regents Exam,Year,Total Tested,Mean Score,Number Scoring Below 65,Percent Scoring Below 65,Number Scoring 65 or Above,Percent Scoring 65 or Above,Number Scoring 80 or Above,Percent Scoring 80 or Above
1,1,4,12,2018,32,90.2,0,0.0,32,100.0,31,96.9
3,1,1,16,2017,16,82.7,0,0.0,16,100.0,10,62.5
4,1,1,8,2015,163,62.9,81,49.7,82,50.3,13,8.0
7,1,1,7,2019,21,81.5,0,0.0,21,100.0,11,52.4
8,1,1,6,2015,128,87.4,2,1.6,126,98.4,112,87.5
...,...,...,...,...,...,...,...,...,...,...,...,...
33024,1,4,2,2017,12,87.7,0,0.0,12,100.0,11,91.7
33026,1,4,12,2017,10,79.7,0,0.0,10,100.0,5,50.0
33027,1,4,2,2018,15,86.9,0,0.0,15,100.0,15,100.0
33029,1,4,2,2019,15,93.2,0,0.0,15,100.0,15,100.0
